# FDO Workflow to create an FDO in a selected data service
Authors: Sven Bingert, Jana Böhm, Triet Doan

In [9]:
!pip3 install doipy==0.5.0 --quiet

In [10]:
from doipy import hello, list_operations, create_fdo, get_connection
import requests

In [11]:
# Read service specific operations
#
def get_operations_from_service(service_PID, registry_URL):
    request_URL = registry_URL + service_PID
    response = requests.get(request_URL)
    result = response.json()
    return result['implementsOperations']

In [12]:
# Read the PID for schema to be able to provide the required input
# for a specific operation
#
def get_operation_schema_PID(operation_PID, registry_URL):
    request_URL = registry_URL + operation_PID
    response = requests.get(request_URL)
    result = response.json()
    inputs_PID = result['inputs']
    return inputs_PID

In [13]:
# Get the schema for the operation
#
def get_operation_schema(schema_PID, registry_URL):
    request_URL = registry_URL + schema_PID
    return requests.get(request_URL).json()

In [14]:
# Resolve a handle based PID
#
def resolve_PID(input):
    Handle_URL='https://hdl.handle.net/api/handles/'
    request_URL = Handle_URL + input
    return requests.get(request_URL).text

## The first step is to find and select a service

### Option a) We search in the service registry and read service information from the registry

In [29]:
# We define the registry URL
# This registry contains, among others, InfoTypes, FDO Types, Operations and Service
#
registry_url='http://typeregistry.lab.pidconsortium.net/objects/'

In [30]:
# We want to search for FDO Services
#
document_type='FDO_Service'

In [31]:
URL=registry_url + '?query=type:' + document_type

In [37]:
# Get all services from the service registry
#
services = requests.get(URL).json()
for service in services['results']:
    print (service['content']['name'] + ' -------- '+ service['content']['serviceType'])
    if  service['content'].get('serviceId',False):
        print(service['content']['serviceId'])

FDO_Manager -------- FDO Manager
FDO_Servive_Registry_Testbed -------- Service Registry
FDO_Testbed_Linkahead -------- Repository
21.T11965/linkahead
FDO_Testbed_Cordra -------- Repository
21.T11967/service
FDO_Testbed_B2Share -------- FDO Manager
21.T11975/service


In [21]:
#  Select a service ID and past here: 
service_id, ip, port = get_connection(service='21.T11967/service')
print(hello(service_id,ip,port))

[{'status': '0.DOIP/Status.001', 'output': {'id': '21.T11967/service', 'type': '0.TYPE/DOIPServiceInfo', 'attributes': {'ipAddress': '0.0.0.0', 'port': 9000, 'protocol': 'TCP', 'protocolVersion': '2.0', 'cordraVersion': {'number': '2.5.2', 'timestamp': '2023-08-29T04:40:13Z', 'id': '405d6d0009c6'}, 'publicKey': {'kty': 'RSA', 'n': 'vWBPsSKco0x0n5_U-mBOJX9mrhQr23FOFbXhTHG4jR30j7r1yZKAQyM0i3MJsXupfwSSf4lMOPRdzkQCM6_rgQQJE7iyTn9ZJrAzSKtUwkbqUzxie_CEOX23N9ibZ8lTbvtvVx0sbNrY9Wg72Q5YRTbR-Gp4RhPx2tzKfGxYclWnApTxtiAMgIEgGHuGne4XrXuqISXno-YGDAsuQ5zfQO6NN9vIKkxDo5lvSEvfJUpaJm4BWigMFsBhrI72imFnNRZGuWkYYmIsOALM16pTU8JAOxtLlC83FeiYfG3zYovOQfTctN3-3jAgNjKn6JRLC9zw6uA0yAL1-2U_DksA3w', 'e': 'AQAB'}}}}]


### Option b) Read service information form the PID system

In [32]:
service_PID = '21.T11967/service'
service_PID_records = json.loads(resolve_PID(service_PID))

In [33]:
# Resolve the PID
#
for val in service_PID_records['values']:
    if val['type'] == "URL":
        service_URL=val['data']['value']
    if val['type'] == "IP":
        service_IP=val['data']['value']
    if val['type'] == "PORT":
        service_PORT=val['data']['value']
print(service_URL, service_IP, service_PORT)

https://cordra.testbed.pid.gwdg.de 141.5.106.77 9000


### Now we can select one of the services, e.g. the cordra object store

In [38]:
# Cordra
#
service_PID='21.T11969/01370800d56a0d897dc1'

In [39]:
# Get Extended Operations, not only the doip basic operations
#
service_operations = get_operations_from_service(service_PID,registry_url)
print('Implemented Operations: ', service_operations)

Implemented Operations:  ['21.T11969/d9ff44c97ea23287483b', '21.T11969/c79f0cd8573a2a2d2b56', '21.T11969/e05b20bacc684f8287b0']


### Select an operation

In [40]:
operation_PID = service_operations[0]

In [41]:
schema_PID = get_operation_schema_PID(operation_PID, registry_url)

### Some manual steps to get the user input data
Those need to be automized to ...

In [26]:
# User credentials can be requested
#
data = {
  "FDO_Service_Ref": service_PID,
  "FDO_Profile_Ref": "21.T11969/141bf451b18a79d0fe66",
  "FDO_Authentication": {
    "username": "admin",
    "password": "***"
  },
  "FDO_Type_Ref": "21.1/thisIsAnFdoType",
  "FDO_Rights_Ref": "21.1/thisIsAnFdoRightsSpecification",
  "FDO_Genre_Ref": "21.1/thisIsAnFdoGenre",
  "FDO_Data_and_Metadata": [
    {
      "data_bitsq": "data_file.txt",
      "metadata_bitsq": "metadata_file.txt",
    }
  ]
}

### Now call create_fdo from the doipy package

In [ ]:
response = create_fdo(data)
print("New created FDO",response)